In [1]:
import pandas as pd
import numpy as np

In [2]:
# Objectif : df avec un film par ligne et le maximum d'infos 
# Etape 1 : Merge de toutes les tables hors film 
# Etape 2 : Rassenblement des colonne similaire 
# Etape 3 : Remplissage des infos manquantes ou suppression des lignes 



In [3]:
# Etape 1 : Merge de toutes les tables hors film 
# Pour la table title akas ont supprime directement les film qui ne sont pas en français (un cinema ne peut pas proposer de film non traduit)
# Il y a aussi des doublons de lignes 
df_title_akas = pd.read_csv("./BD/title.akas.tsv.gz", compression = 'gzip', na_values=['\\N'], sep = '\t')
df_title_akas = df_title_akas[(df_title_akas['region'] == 'FR') | (df_title_akas['language'] == 'fr')]
df_title_akas = df_title_akas.drop_duplicates(subset='titleId', keep='first')

# Pour la table title basics on ne récupère que les films (titleType == 'movie')
df_title_basics = pd.read_csv("BD/title.basics.tsv.gz", compression = 'gzip', na_values=['\\N'], sep = '\t')
df_title_basics = df_title_basics[(df_title_basics['titleType'] == 'movie')]

# On ne s'occupe pas de title crew pour le moment car on s'en occupera dans la gestion des acteurs
# On ne s'occupe pas de title episode car elle sert pour les series (vu dans le doc) 
# On ne s'occupe pas de title principals car on s'en occupera dans la gestion des acteurs 

# Pour la table title rating on récupère tout mais on attend le même nombre que précédement 
df_title_rating = pd.read_csv("BD/title.ratings.tsv.gz", compression = 'gzip', na_values=['\\N'], sep = '\t')

# On ne s'occupe pas de name basics car on s'en occupera dans la gestion des acteurs 

# Pour la table TMBD FULL on récupère tout 
df_tmdb_full = pd.read_csv("./BD/tmdb_full.csv")
df_tmdb_full['fr?'] = df_tmdb_full['spoken_languages'].apply(lambda x: 'fr' in x)
df_tmdb_full = df_tmdb_full[df_tmdb_full['fr?']]

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_4141/527731522.py:4: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_title_akas = pd.read_csv("./BD/title.akas.tsv.gz", compression = 'gzip', na_values=['\\N'], sep = '\t')
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_4141/527731522.py:9: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_title_basics = pd.read_csv("BD/title.basics.tsv.gz", compression = 'gzip', na_values=['\\N'], sep = '\t')
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_4141/527731522.py:22: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tmdb_full = pd.read_csv("./BD/tmdb_full.csv")


In [4]:
# Merge entre df_title_akas et df_title_basics (attendu df_merge1 >= df_title_akas ou df_title_basics)
df_merge1 = pd.merge(df_title_akas,
                     df_title_basics,
                     how = 'right',
                     left_on = df_title_akas['titleId'],
                     right_on = df_title_basics['tconst'])
df_merge1['tconst'] = df_merge1['key_0']
df_merge1 = df_merge1.drop(['key_0', 'titleId'], axis = 1)

In [5]:
# Merge entre df_merge1 et df_title_rating (attendu df_merge2 = df_merge1 (en ligne))
df_merge2 = pd.merge(df_merge1,
                     df_title_rating,
                     how = 'left',
                     on = 'tconst')

In [6]:
# Merge entre df_merge2 et df_tmdb_full (attendu df_merge3 >= df_merge1 ou df_tmdb_full (en ligne))
df_merge3 = pd.merge(df_merge2,
                     df_tmdb_full,
                     how = 'outer',
                     left_on = df_merge2['tconst'],
                     right_on = df_tmdb_full['imdb_id'])

In [7]:
# Etape 2 : Rassenblement des colonne similaire (lors d'un choix on priorise arbitrairement la table TMBD)
# tconst / imdb_id / id sont à retirer
# On le titre title_y / title_x / primaryTitle / originalTitle
# Le genre est à gérer genres_x / genres_y
# La note averageRating et vote_average
# Le nombre de vote vote_count et numVotes

In [8]:
# tconst / imdb_id / id sont à retirer
df_merge3['tconst'] = df_merge3['key_0']
df_merge3 = df_merge3.drop(['key_0', 'imdb_id', 'id'], axis = 1)

In [9]:
# On le titre title_y / title_x / primaryTitle / originalTitle
# Créer une nouvelle colonne 'titre' en priorisant les colonnes dans l'ordre
df_merge3['titre'] = df_merge3['title_y'].fillna(df_merge3['title_x']).fillna(df_merge3['primaryTitle']).fillna(df_merge3['originalTitle'])
df_merge3 = df_merge3.drop(['title_y', 'title_x', 'primaryTitle', 'originalTitle'], axis = 1)


In [10]:
# Le genre est à gérer genres_x / genres_y
df_merge3['genres_y'] = df_merge3['genres_y'].apply(lambda x: x if (type(x) == list) else [])
df_merge3['genres_x_list'] = df_merge3['genres_x'].fillna('').str.split(', ')
df_merge3['genre'] = df_merge3.apply(lambda row: list(set(row['genres_y'] + row['genres_x_list'])),axis=1)
df_merge3 = df_merge3.drop(['genres_x_list', 'genres_x', 'genres_y'], axis = 1)
df_merge3['genre'] = df_merge3['genre'].apply(lambda x: x if x != [] else np.nan)


In [11]:
# La note averageRating et vote_average et le nombre de vote vote_count et numVotes
df_merge4 = df_merge3.copy()
df_merge4 = df_merge4[~((df_merge4['vote_average'].isna()) & (df_merge4['averageRating'].isna()))]
df_merge4 = df_merge4[~((df_merge4['vote_average'] == 0) & (df_merge4['averageRating'] == 0))]
df_merge4 = df_merge4[~((df_merge4['vote_average'] == 0) & (df_merge4['averageRating'].isna()))]
df_merge4 = df_merge4[~((df_merge4['vote_average'].isna()) & (df_merge4['averageRating'] == 0))]

df_merge4['vote_average'] = round(df_merge4['vote_average'], 2)

df_a_garder_1 = df_merge4[df_merge4['averageRating'] == df_merge4['vote_average']]
df_a_travailler_1 = df_merge4[~(df_merge4['averageRating'] == df_merge4['vote_average'])]

df_a_garder_2 = df_a_travailler_1[(df_a_travailler_1['averageRating'].isna()) | (df_a_travailler_1['vote_average'].isna()) |(df_a_travailler_1['vote_average'] == 0) | (df_a_travailler_1['averageRating'] == 0)]

df_a_travailler_2 = df_a_travailler_1[~((df_a_travailler_1['averageRating'].isna()) | (df_a_travailler_1['vote_average'].isna()) |(df_a_travailler_1['vote_average'] == 0) | (df_a_travailler_1['averageRating'] == 0))]

df_a_travailler_2['difference_vote'] = abs(df_a_travailler_2['averageRating'] - df_a_travailler_2['vote_average'])
df_a_garder_3 = df_a_travailler_2[df_a_travailler_2['difference_vote'] <= 1]
df_a_travailler_3 = df_a_travailler_2[df_a_travailler_2['difference_vote'] > 1]

df_a_travailler_3['vote_exact'] = ((df_a_travailler_3['vote_count'] * df_a_travailler_3['vote_average']) + (df_a_travailler_3['numVotes'] * df_a_travailler_3['averageRating'])) / (df_a_travailler_3['numVotes'] + df_a_travailler_3['vote_count'])
df_a_travailler_3['vote_exact'] = round(df_a_travailler_3['vote_exact'], 1)

df_a_garder_4 = df_a_travailler_3

df_a_garder_1['vote_exact'] = df_a_garder_1[['averageRating', 'vote_average']].max(axis=1)
df_a_garder_2['vote_exact'] = df_a_garder_2[['averageRating', 'vote_average']].max(axis=1)
df_a_garder_3['vote_exact'] = df_a_garder_3[['averageRating', 'vote_average']].mean(axis=1)

df_a_garder_1['nombre_de_votes'] = df_a_garder_1['vote_count']
df_a_garder_2['nombre_de_votes'] = df_a_garder_2['vote_count']
df_a_garder_3['nombre_de_votes'] = df_a_garder_3['vote_count']
df_a_garder_4['nombre_de_votes'] = df_a_garder_4['vote_count'] + df_a_garder_4['numVotes']

df_merge4 = pd.concat([df_a_garder_1, df_a_garder_2, df_a_garder_3, df_a_garder_4])

df_merge4['note'] = round(df_merge4['vote_exact'], 2)
df_merge4 = df_merge4.drop(['vote_count', 'averageRating', 'vote_average', 'numVotes', 'difference_vote', 'vote_exact'], axis=1)



/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_4141/1535123634.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_travailler_2['difference_vote'] = abs(df_a_travailler_2['averageRating'] - df_a_travailler_2['vote_average'])
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_4141/1535123634.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_travailler_3['vote_exact'] = ((df_a_travailler_3['vote_count'] * df_a_travailler_3['vote_average']) + (df_a_travailler_3['numVotes'] * d

In [12]:
# La -18 à fusionner entre isAdult (0 ou 1) et adult (bool) --> adult True c'est film pour adulte
df_merge5 = df_merge4.copy()
df_merge5['adult'] = (df_merge5['isAdult'] == 1) | df_merge5['adult']
df_merge5 = df_merge5.drop(['isAdult'], axis=1)

In [13]:
# release_date et startYear à fusionner 
df_merge5['release_date'] = pd.to_datetime(df_merge5['release_date'], errors='coerce')
df_merge5['release_date'] = df_merge5['release_date'].dt.year
df_merge5 = df_merge5[~((df_merge5['release_date'].isna()) & (df_merge5['startYear'].isna()))]
df_merge5 = df_merge5[~((df_merge5['release_date'] == 0) & (df_merge5['startYear'] == 0))]
df_merge5 = df_merge5[~((df_merge5['release_date'] == 0) & (df_merge5['startYear'].isna()))]
df_merge5 = df_merge5[~((df_merge5['release_date'].isna()) & (df_merge5['startYear'] == 0))]

df_merge5['startYear'].fillna(0, inplace = True)
df_merge5['release_date'].fillna(0, inplace = True)
df_merge5['startYear'] = df_merge5['startYear'].astype(int)
df_merge5['release_date'] = df_merge5['release_date'].astype(int)

df_merge5['Date_sortie'] = df_merge5[['startYear', 'release_date']].max(axis = 1)
df_merge5 = df_merge5.drop(['startYear','release_date'], axis=1)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_4141/1768297304.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_merge5['startYear'].fillna(0, inplace = True)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_4141/1768297304.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alway

In [14]:
# runtime et runtimeMinutes à fusionner 
# Clean du Merge

df_merge5['runtimeMinutes'] = pd.to_numeric(df_merge5['runtimeMinutes'])
df_merge5 = df_merge5[~((df_merge5['runtimeMinutes'].isna()) & (df_merge5['runtime'].isna()))]
df_merge5 = df_merge5[~((df_merge5['runtimeMinutes'] == 0) & (df_merge5['runtime'] == 0))]
df_merge5 = df_merge5[~((df_merge5['runtimeMinutes'] == 0) & (df_merge5['runtime'].isna()))]
df_merge5 = df_merge5[~((df_merge5['runtimeMinutes'].isna()) & (df_merge5['runtime'] == 0))]

df_merge5['difference_runtime'] = abs(df_merge5['runtime'] - df_merge5['runtimeMinutes'])

# On met de côté là où les 2 valeurs sont identiques
df_a_garder_1 = df_merge5[df_merge5['runtimeMinutes'] == df_merge5['runtime']].copy()
df_a_garder_1['runtime_exact'] = df_a_garder_1['runtimeMinutes']

# On met de côté là où les 2 valeurs ne sont pas identiques
df_a_travailler_1 = df_merge5[df_merge5['runtimeMinutes'] != df_merge5['runtime']].copy()

# On conserve la donnée existante
df_a_garder_2 = df_a_travailler_1[(df_a_travailler_1['runtimeMinutes'].isna()) | (df_a_travailler_1['runtime'].isna()) | (df_a_travailler_1['runtime'] == 0) | (df_a_travailler_1['runtimeMinutes'] == 0)].copy()

# On conserve la valeur max de runtime quand les deux sont là
df_a_garder_2['runtime_exact'] = df_a_garder_2[['runtimeMinutes', 'runtime']].max(axis=1)


df_merge5 = pd.concat([df_a_garder_1, df_a_garder_2])
df_merge5['durée'] = df_merge5['runtime_exact']
df_merge5 = df_merge5.drop(['difference_runtime', 'runtime', 'runtimeMinutes', 'runtime_exact'], axis=1)


In [15]:
# tagline et overview à fusionner ???

In [16]:
# Les films dont le status est cancelled à retirer 

In [17]:
df_merge6 = df_merge5.copy()

In [18]:
df_title_akas, df_title_basics, df_title_rating, df_tmdb_full, df_merge1, df_merge2, df_merge3, df_merge4, df_merge5, df_a_garder_1, df_a_garder_2, df_a_garder_3, df_a_garder_4, df_a_travailler_1, df_a_travailler_2, df_a_travailler_3 = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [19]:
df_merge6 = df_merge6.drop(['ordering', 'language', 'types', 'attributes', 'titleType', 'endYear', 'original_title', 'spoken_languages', 'video', 'fr?', 'titleType', 'status'], axis = 1)

In [20]:
# Etape 3 : Remplissage des infos manquantes ou suppression des lignes trop peu utilisable

In [21]:
# Etape 4 : Préparation du fichier pour rajouter les acteur et autres
df_title_principals = pd.read_csv("BD/title.principals.tsv.gz", compression = 'gzip', na_values=['\\N'], sep = '\t')
df_name_basics = pd.read_csv("BD/name.basics.tsv.gz", compression = 'gzip', na_values=['\\N'], sep = '\t')

In [22]:
df_acteur = pd.merge(df_title_principals,
                     df_name_basics,
                     how = 'left',
                     on = 'nconst')
df_acteur = df_acteur.dropna(subset=['primaryName'])

# Crash car trop de ligne. Solution drop les colonne et les lignes pas intéressante de df_title_principals et df_name_basics

In [23]:
df_title_principals, df_name_basics = 0, 0

In [24]:
df_acteur = df_acteur.drop(['ordering', 'job', 'characters', 'primaryProfession', 'knownForTitles'], axis=1)

In [25]:
# Etape 4 : Merge entre la table des films et des acteur 

df = pd.merge(df_acteur,
              df_merge6,
              how = 'left',
              on = 'tconst')

In [27]:
df_acteur, df_merge6 = 0, 0

In [29]:
df.to_csv('data.csv.gz', index=False, compression='gzip')